In [ ]:
'''
1. データセット用フォルダー「data」と
   学習済みの重み用フォルダー「weight」の作成
'''
import os

# 「data」フォルダーを
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# 「weights」フォルダーを作成
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

In [3]:
%%time
'''
2. VOC2012のデータセットを
   Visual Object Classes Challenge 2012のサイトからダウンロード
'''
import urllib.request
import tarfile

# VOCデータセットのダウンロード先のURL
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
# data_dirにファイル名を連結してファイルパスを作成
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar") 

if not os.path.exists(target_path):
    # urlにアクセスしてファイルパスtarget_pathとしてダウンロード
    urllib.request.urlretrieve(url, target_path)
    # ダウンロードしたtar形式ファイルを読み込む
    tar = tarfile.TarFile(target_path)
    tar.extractall(data_dir)  # tarファイルを解凍
    tar.close()               # tarファイルをクローズ
    

CPU times: user 9.07 s, sys: 12.3 s, total: 21.4 s
Wall time: 1min 22s


In [4]:
%%time

url = 'https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth'
target_path = os.path.join(weights_dir, 'vgg16_reducedfc.pth')

if not os.path.exists(target_path):
  urllib.request.urlretrieve(url, target_path)

CPU times: user 185 ms, sys: 212 ms, total: 397 ms
Wall time: 1.38 s


In [5]:
%%time

url = 'https://s3.amazonaws.com/amdegroot-models/ssd300_mAP_77.43_v2.pth'
target_path = os.path.join(weights_dir, 'ssd300_mAP_77.43_v2.pth')

if not os.path.exists(target_path):
  urllib.request.urlretrieve(url, target_path)

CPU times: user 256 ms, sys: 224 ms, total: 480 ms
Wall time: 1.92 s


In [5]:
from google.colab import drive
drive.mount('/content/drive')

import sys
ROOT_PATH = '/content/drive/MyDrive/MyColab'
sys.path.append(ROOT_PATH)

from voc import make_filepath_list

rootpath = './data/VOCdevkit/VOC2012/'
tr_img, tr_anno, \
    val_img, val_anno = make_filepath_list(rootpath)

print(tr_img[0])
print(tr_anno[0])
print(val_img[0])
print(val_anno[0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored

In [ ]:

import cv2
from voc import GetBBoxAndLabel

voc_labels = [
    'aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow',
    'diningtable','dog','horse','motorbikes','person','pottedplant','sheep','sofa','train','tvmonitor',
]

annotation = GetBBoxAndLabel(voc_labels)

index = 1
img_path = val_img[index]
